In [1]:
import requests
import json
import pandas as pd
import numpy as np

!pip install quandl
import quandl

You should consider upgrading via the '/Users/Lex/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
#S&P500 Data Call

url = "https://alpha-vantage.p.rapidapi.com/query"

querystring = {"function":"TIME_SERIES_DAILY","symbol":"SPY","outputsize":"full","datatype":"json"}
# swap out compact for full to get 20 year history

headers = {
    'x-rapidapi-key': "85073711famsh14c401fb1c62dc5p138f19jsnd902d78d96ca",
    'x-rapidapi-host': "alpha-vantage.p.rapidapi.com"
    }

results = requests.request("GET", url, headers=headers, params=querystring)

#print(results.text)

In [3]:
# S&P 500 Data

sp500=pd.DataFrame()
sp500 = pd.DataFrame.from_dict(results.json()["Time Series (Daily)"])

sp500 = sp500.T
sp500.columns = ['open', 'high', 'low', 'close', 'volume']

sp500['prevclose'] = sp500.close.shift(1).astype(float)
sp500['close'] = sp500['close'].astype(float)

sp500['pct_change_in_close'] = (sp500['close'] - sp500['prevclose']) / sp500['prevclose']*100

sp500.head()

,open,high,low,close,volume,prevclose,pct_change_in_close
1999-11-01,136.5000,137.0000,135.5625,135.5625,4006500,NaN,NaN
1999-11-02,135.9687,137.2500,134.5937,134.5937,6516900,135.5625,-0.714652
1999-11-03,136.0000,136.3750,135.1250,135.5000,7222300,134.5937,0.673360
1999-11-04,136.7500,137.3593,135.7656,136.5312,7907500,135.5000,0.761033
1999-11-05,138.6250,139.1093,136.7812,137.8750,7431500,136.5312,0.984244


In [4]:
# Pulling the Fed_Rate

fed_rate = quandl.get("FRED/DFF", authtoken="NPExo1wnSkPZTDxv2bCR")

fed_rate['Prev_Rate'] = fed_rate.Value.shift(1).astype(float)
fed_rate['Value'] = fed_rate['Value'].astype(float)

fed_rate['pct_change_in_fed_rate'] = (fed_rate['Value'] - fed_rate['Prev_Rate']) / fed_rate['Value']*100

fed_rate.head()

,Value,Prev_Rate,pct_change_in_fed_rate
Date,,,
1954-07-01,1.13,NaN,NaN
1954-07-02,1.25,1.13,9.600000
1954-07-03,1.25,1.25,0.000000
1954-07-04,1.25,1.25,0.000000
1954-07-05,0.88,1.25,-42.045455


In [5]:
# Pulling US Treasuries

ten_year = quandl.get("USTREASURY/REALYIELD", authtoken="NPExo1wnSkPZTDxv2bCR")
ten_year.columns = ['five_yr', 'seven_yr', 'ten_yr', 'twenty_yr', 'thirty_yr']



ten_year['prev_10_yr'] = ten_year.ten_yr.shift(1).astype(float)
ten_year['ten_yr'] = ten_year['ten_yr'].astype(float)

ten_year['pct_change_in_10_yr'] = (ten_year['ten_yr'] - ten_year['prev_10_yr']) / ten_year['ten_yr']*100

ten_year.head()

,five_yr,seven_yr,ten_yr,twenty_yr,thirty_yr,prev_10_yr,pct_change_in_10_yr
Date,,,,,,,
2003-01-02,1.75,2.24,2.43,NaN,NaN,NaN,NaN
2003-01-03,1.75,2.25,2.43,NaN,NaN,2.43,0.000000
2003-01-06,1.79,2.27,2.46,NaN,NaN,2.43,1.219512
2003-01-07,1.76,2.24,2.42,NaN,NaN,2.46,-1.652893
2003-01-08,1.68,2.14,2.29,NaN,NaN,2.42,-5.676856


In [6]:
sp500_and_fed_rate = pd.merge(sp500, fed_rate, left_index=True, right_index=True)
sp500_and_fed_rate_and_ten_year = pd.merge(sp500_and_fed_rate, ten_year, left_index=True, right_index=True)

combined = sp500_and_fed_rate_and_ten_year.reset_index()

combined.columns = ['Date', 'open', 'high', 'low', 'close', 'volume', 'prevclose', 'pct_change_in_close', 'fed_rate', 'prev_fed_rate', 'pct_change_in_fed_rate', '5_yr', '7_yr', '10_yr', '20_yr', '30_yr', 'prev_10_yr', 'pct_change_in_10_yr']

combined['Date'] = pd.to_datetime(combined['Date'], format='%Y%m%d', errors='ignore')
combined.head()

,Date,open,high,low,close,volume,prevclose,pct_change_in_close,fed_rate,prev_fed_rate,pct_change_in_fed_rate,5_yr,7_yr,10_yr,20_yr,30_yr,prev_10_yr,pct_change_in_10_yr
0,2003-01-02,88.8500,91.3000,88.5400,91.07,44516300,88.23,3.218860,1.30,1.16,10.769231,1.75,2.24,2.43,NaN,NaN,NaN,NaN
1,2003-01-03,90.9100,91.3800,90.5000,91.35,32222600,91.07,0.307456,1.12,1.30,-16.071429,1.75,2.25,2.43,NaN,NaN,2.43,0.000000
2,2003-01-06,91.2400,93.4900,91.1700,92.96,40984500,91.35,1.762452,1.22,1.12,8.196721,1.79,2.27,2.46,NaN,NaN,2.43,1.219512
3,2003-01-07,92.9000,93.3700,92.2000,92.73,38640400,92.96,-0.247418,1.20,1.22,-1.666667,1.76,2.24,2.42,NaN,NaN,2.46,-1.652893
4,2003-01-08,92.2000,92.4000,91.0500,91.39,38702200,92.73,-1.445056,1.29,1.20,6.976744,1.68,2.14,2.29,NaN,NaN,2.42,-5.676856


In [7]:
combined.dtypes

Date                      datetime64[ns]
open                              object
high                              object
low                               object
close                            float64
volume                            object
prevclose                        float64
pct_change_in_close              float64
fed_rate                         float64
prev_fed_rate                    float64
pct_change_in_fed_rate           float64
5_yr                             float64
7_yr                             float64
10_yr                            float64
20_yr                            float64
30_yr                            float64
prev_10_yr                       float64
pct_change_in_10_yr              float64
dtype: object

In [8]:
# Pulling in Fed dates

df_dates = pd.read_csv('FOMC_meeting_dates_els.csv')
df_dates['Date'] = pd.to_datetime(df_dates['Date'], format='%Y%m%d', errors='ignore')

#df_dates = df_dates.set_index('Date')

df_dates.head()

FileNotFoundError: File b'FOMC_meeting_dates_els.csv' does not exist

In [ ]:
df_dates.dtypes

Date               datetime64[ns]
Type                       object
PressConference             int64
dtype: object

In [ ]:
df_final =  pd.merge(df_dates, combined, left_on='Date', right_on='Date')


In [ ]:
df_final

,Date,Type,PressConference,open,high,low,close,volume,prevclose,pct_change_in_close,fed_rate,prev_fed_rate,pct_change_in_fed_rate,5_yr,7_yr,10_yr,20_yr,30_yr,prev_10_yr,pct_change_in_10_yr
0,2010-01-27,Meeting,0,109.1700,110.0800,108.3300,109.83,271482263,108.57,1.160542,0.12,0.12,0.000000,0.47,0.89,1.35,2.00,NaN,1.31,2.962963
1,2010-03-16,Meeting,0,115.8100,116.5200,115.4900,116.41,168561421,117.10,-0.589240,0.20,0.20,0.000000,0.45,0.98,1.42,1.92,2.11,1.46,-2.816901
2,2010-04-28,Meeting,0,119.0500,119.6800,118.2700,119.38,300539227,120.86,-1.224557,0.20,0.20,0.000000,0.53,1.00,1.40,1.78,1.97,1.37,2.142857
3,2010-06-23,Meeting,0,109.6400,110.0300,108.4800,109.23,254527875,107.42,1.684975,0.17,0.18,-5.882353,0.29,0.70,1.19,1.64,1.71,1.20,-0.840336
4,2010-08-10,Meeting,0,112.0300,112.9800,111.3700,112.38,242692534,109.30,2.817932,0.18,0.18,0.000000,-0.02,0.39,0.95,1.63,1.78,1.04,-9.473684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,2020-04-08,Minutes,1,267.9600,276.0000,265.2500,274.03,152670223,278.20,-1.498922,0.05,0.05,0.000000,-0.40,-0.42,-0.42,-0.24,-0.04,-0.42,-0.000000
168,2020-05-20,Minutes,1,295.8200,297.8700,295.5700,296.93,85279592,294.88,0.695198,0.05,0.05,0.000000,-0.49,-0.49,-0.48,-0.29,-0.10,-0.45,6.250000
169,2020-07-01,Minutes,1,309.5700,311.8900,309.0700,310.52,71910372,312.23,-0.547673,0.08,0.08,0.000000,-0.86,-0.79,-0.68,-0.38,-0.15,-0.68,-0.000000
170,2020-08-19,Minutes,1,339.0500,339.6100,336.6200,337.23,68054244,338.28,-0.310394,0.09,0.09,0.000000,-1.26,-1.15,-0.99,-0.60,-0.33,-1.01,-2.020202


In [ ]:
#df_final_trimmed = df_final[['Date', 'Type', 'PressConference', 'pct_change_in_close', 'pct_change_in_fed_rate', 'pct_change_in_10_yr']]
#df_final_trimmed

,Date,Type,PressConference,pct_change_in_close,pct_change_in_fed_rate,pct_change_in_10_yr
0,2010-01-27,Meeting,0,1.160542,0.000000,2.962963
1,2010-03-16,Meeting,0,-0.589240,0.000000,-2.816901
2,2010-04-28,Meeting,0,-1.224557,0.000000,2.142857
3,2010-06-23,Meeting,0,1.684975,-5.882353,-0.840336
4,2010-08-10,Meeting,0,2.817932,0.000000,-9.473684
...,...,...,...,...,...,...
167,2020-04-08,Minutes,1,-1.498922,0.000000,-0.000000
168,2020-05-20,Minutes,1,0.695198,0.000000,6.250000
169,2020-07-01,Minutes,1,-0.547673,0.000000,-0.000000
170,2020-08-19,Minutes,1,-0.310394,0.000000,-2.020202


In [ ]:
# create a list of our conditions
conditions = [
    # S&P500 is positive

    # Positive, Negative, Positive = 1 - 1 + 1 = 1
    ((df_final['pct_change_in_close'] >= 0) & (df_final['pct_change_in_fed_rate'] < 0) & (df_final['pct_change_in_10_yr'] >= 0)),

    # Positive, Negative, Negative = 1 - 1 - 1 = -1
    ((df_final['pct_change_in_close'] >= 0) & (df_final['pct_change_in_fed_rate'] < 0) & (df_final['pct_change_in_10_yr'] < 0)),

    # Positive, Neutral, #Positive = 1 + 0 + 1 = 2
    ((df_final['pct_change_in_close'] >= 0) & (df_final['pct_change_in_fed_rate'] == 0) & (df_final['pct_change_in_10_yr'] >= 0)),

    # Positive, Neutral, #Negative = 1 + 0 - 1 = 0
    ((df_final['pct_change_in_close'] >= 0) & (df_final['pct_change_in_fed_rate'] == 0) & (df_final['pct_change_in_10_yr'] < 0)),

    # Positive, Positve, #Positive = 1 + 1 + 1 = 3
    ((df_final['pct_change_in_close'] >= 0) & (df_final['pct_change_in_fed_rate'] > 0) & (df_final['pct_change_in_10_yr'] >= 0)),

    # Positive, Positive, #Negative = 1 + 1 - 1 = 2
    ((df_final['pct_change_in_close'] >= 0) & (df_final['pct_change_in_fed_rate'] > 0) & (df_final['pct_change_in_10_yr'] < 0)),

    #S&P 500 is negative

    # Negatve, Negative, Positive = -1 - 1 + 1 = -1
    ((df_final['pct_change_in_close'] < 0) & (df_final['pct_change_in_fed_rate'] < 0) & (df_final['pct_change_in_10_yr'] >= 0)),

    # Negatve, Negative, Negative = -1 - 1 - 1 = -3
    ((df_final['pct_change_in_close'] < 0) & (df_final['pct_change_in_fed_rate'] < 0) & (df_final['pct_change_in_10_yr'] < 0)),

    # Negatve, Neutral, #Positive = -1 + 0 + 1 = 0
    ((df_final['pct_change_in_close'] < 0) & (df_final['pct_change_in_fed_rate'] == 0) & (df_final['pct_change_in_10_yr'] >= 0)),

    # Negatve, Neutral, #Negative = -1 + 0 - 1 = -2
    ((df_final['pct_change_in_close'] < 0) & (df_final['pct_change_in_fed_rate'] == 0) & (df_final['pct_change_in_10_yr'] < 0)),

    # Negatve, Positve, #Positive = -1 + 1 + 1 = 1
    ((df_final['pct_change_in_close'] < 0) & (df_final['pct_change_in_fed_rate'] > 0) & (df_final['pct_change_in_10_yr'] >= 0)),

    # Negatve, Positive, #Negative = -1 + 1 - 1 = -1
    ((df_final['pct_change_in_close'] < 0) & (df_final['pct_change_in_fed_rate'] > 0) & (df_final['pct_change_in_10_yr'] < 0)),
    
    ]

# create a list of the values we want to assign for each condition
values = ['1', '-1', '2', '0', '3', '2', '-1', '-3', '0', '-2', '1', '-1']

# create a new column and use np.select to assign values to it using our lists as arguments
df_final['label'] = np.select(conditions, values)


In [ ]:
df_final

,Date,Type,PressConference,open,high,low,close,volume,prevclose,pct_change_in_close,fed_rate,prev_fed_rate,pct_change_in_fed_rate,5_yr,7_yr,10_yr,20_yr,30_yr,prev_10_yr,pct_change_in_10_yr,label
0,2010-01-27,Meeting,0,109.1700,110.0800,108.3300,109.83,271482263,108.57,1.160542,0.12,0.12,0.000000,0.47,0.89,1.35,2.00,NaN,1.31,2.962963,2
1,2010-03-16,Meeting,0,115.8100,116.5200,115.4900,116.41,168561421,117.10,-0.589240,0.20,0.20,0.000000,0.45,0.98,1.42,1.92,2.11,1.46,-2.816901,-2
2,2010-04-28,Meeting,0,119.0500,119.6800,118.2700,119.38,300539227,120.86,-1.224557,0.20,0.20,0.000000,0.53,1.00,1.40,1.78,1.97,1.37,2.142857,0
3,2010-06-23,Meeting,0,109.6400,110.0300,108.4800,109.23,254527875,107.42,1.684975,0.17,0.18,-5.882353,0.29,0.70,1.19,1.64,1.71,1.20,-0.840336,-1
4,2010-08-10,Meeting,0,112.0300,112.9800,111.3700,112.38,242692534,109.30,2.817932,0.18,0.18,0.000000,-0.02,0.39,0.95,1.63,1.78,1.04,-9.473684,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,2020-04-08,Minutes,1,267.9600,276.0000,265.2500,274.03,152670223,278.20,-1.498922,0.05,0.05,0.000000,-0.40,-0.42,-0.42,-0.24,-0.04,-0.42,-0.000000,0
168,2020-05-20,Minutes,1,295.8200,297.8700,295.5700,296.93,85279592,294.88,0.695198,0.05,0.05,0.000000,-0.49,-0.49,-0.48,-0.29,-0.10,-0.45,6.250000,2
169,2020-07-01,Minutes,1,309.5700,311.8900,309.0700,310.52,71910372,312.23,-0.547673,0.08,0.08,0.000000,-0.86,-0.79,-0.68,-0.38,-0.15,-0.68,-0.000000,0
170,2020-08-19,Minutes,1,339.0500,339.6100,336.6200,337.23,68054244,338.28,-0.310394,0.09,0.09,0.000000,-1.26,-1.15,-0.99,-0.60,-0.33,-1.01,-2.020202,-2


In [ ]:
df_final.groupby(['label']).mean()

,PressConference,close,prevclose,pct_change_in_close,fed_rate,prev_fed_rate,pct_change_in_fed_rate,5_yr,7_yr,10_yr,20_yr,30_yr,prev_10_yr,pct_change_in_10_yr
label,,,,,,,,,,,,,,
-1,0.291667,163.386667,163.555000,0.016297,0.317917,0.318333,-1.140591,-0.262083,0.052500,0.374583,0.903333,1.090435,0.392083,-inf
-2,0.612903,223.223871,224.419677,-0.538830,0.765484,0.765484,0.000000,-0.050000,0.170000,0.340000,0.715806,0.917097,0.372258,-21.509646
-3,0.250000,142.092500,142.507500,-0.325816,0.102500,0.112500,-11.458333,-0.347500,0.160000,0.560000,1.162500,1.437500,0.607500,-18.566630
0,0.591549,214.013803,213.653944,0.195143,0.670563,0.670563,0.000000,-0.075352,0.170282,0.358592,0.750282,0.957746,0.361268,-1.790039
1,0.700000,204.360000,203.572000,0.594954,0.839000,0.839000,-1.047297,0.009000,0.239000,0.462000,0.864000,1.062000,0.432000,25.852772
2,0.482759,209.521379,208.247586,0.647912,0.443103,0.442414,0.260673,-0.338276,-0.078276,0.147931,0.598276,0.791786,0.133103,11.463014
3,0.666667,241.373333,233.703333,2.680712,0.820000,0.806667,7.827646,0.253333,0.433333,0.603333,0.916667,1.073333,0.573333,11.553128
